<a href="https://colab.research.google.com/github/Sakinat-Folorunso/CMP_805_Advanced_Programming_Languages/blob/main/notebooks/CMP805_Week2_PH_Python_Colab_(1).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# CMP805 — Week 2 Practical (Python, Colab)
**Topic:** Syntax & parsing; binding, scope, and environments**  
**Course:** Advanced Programming Languages (M.Sc.), OOU — CMP805

**Instructor:** **DR SAKINAT FOLORUNSO – ASSOCIATE PROFESSOR OF AI SYSTEMS AND FAIR DATA**  
**Department:** **COMPUTER SCIENCES, OLABISI ONABANJO UNIVERSITY, AGO‑IWOYE, OGUN STATE, NIGERIA**

> This lab builds a tiny **tokenizer + parser** for the Week‑1 AST and sketches an interpreter.

### Learning goals (≈60 minutes)
- Implement a simple **tokenizer** and **recursive‑descent parser** with operator precedence.  
- Parse `if … then … else …` and `let x = … in …` forms.  
- Run an **interpreter skeleton** and unit tests.

In [ ]:
# 🧑‍🎓 Student info
STUDENT_NAME = "Type your full name here"
STUDENT_ID   = "Matric/ID here"
print("Student:", STUDENT_NAME, "| ID:", STUDENT_ID)

In [ ]:
# ✅ Environment check (Python 3.10+)
import sys
major, minor = sys.version_info[:2]
assert (major, minor) >= (3, 10), f"Need Python 3.10+, found {major}.{minor}"
print(f"Python {major}.{minor} OK — ready to parse.")

In [ ]:
# Reuse Week-1 AST types
from __future__ import annotations
from dataclasses import dataclass
from typing import Union, Dict, List

@dataclass(frozen=True) class Int:  n: int
@dataclass(frozen=True) class Bool: b: bool
@dataclass(frozen=True) class Var:  x: str
@dataclass(frozen=True) class Add:  a: "Expr"; b: "Expr"
@dataclass(frozen=True) class Sub:  a: "Expr"; b: "Expr"
@dataclass(frozen=True) class Mul:  a: "Expr"; b: "Expr"
@dataclass(frozen=True) class Eq:   a: "Expr"; b: "Expr"
@dataclass(frozen=True) class If:   c: "Expr"; t: "Expr"; e: "Expr"
@dataclass(frozen=True) class Let:  x: str; e1: "Expr"; e2: "Expr"
Expr = Union[Int, Bool, Var, Add, Sub, Mul, Eq, If, Let]

def pretty(e: Expr) -> str:
    match e:
        case Int(n):  return str(n)
        case Bool(b): return str(b).lower()
        case Var(x):  return x
        case Add(a,b):return f"({pretty(a)} + {pretty(b)})"
        case Sub(a,b):return f"({pretty(a)} - {pretty(b)})"
        case Mul(a,b):return f"({pretty(a)} * {pretty(b)})"
        case Eq(a,b): return f"({pretty(a)} == {pretty(b)})"
        case If(c,t,e):return f"if {pretty(c)} then {pretty(t)} else {pretty(e)}"
        case Let(x,e1,e2):return f"let {x} = {pretty(e1)} in {pretty(e2)}"

In [ ]:
# -------------------------------------
# Tokenizer
# -------------------------------------
import re

TOK_SPEC = [
    ("INT",   r"\d+"),
    ("TRUE",  r"true\b"),
    ("FALSE", r"false\b"),
    ("LET",   r"let\b"),
    ("IN",    r"in\b"),
    ("IF",    r"if\b"),
    ("THEN",  r"then\b"),
    ("ELSE",  r"else\b"),
    ("EQ",    r"=="),
    ("ID",    r"[A-Za-z_][A-Za-z0-9_]*"),
    ("SKIP",  r"[ \t]+"),
    ("NEWL",  r"[\r\n]+"),
    ("LP",    r"\("),
    ("RP",    r"\)"),
    ("PLUS",  r"\+"),
    ("MINUS", r"\-"),
    ("TIMES", r"\*"),
    ("ASSIGN",r"="),
]

TOK_REGEX = re.compile("|".join(f"(?P<{name}>{pat})" for name, pat in TOK_SPEC))

class Tok:
    def __init__(self, typ, val, pos):
        self.typ = typ; self.val = val; self.pos = pos
    def __repr__(self): return f"Tok({self.typ},{self.val!r},{self.pos})"

def tokenize(s: str):
    pos = 0
    for m in TOK_REGEX.finditer(s):
        kind = m.lastgroup; text = m.group()
        if kind in ("SKIP","NEWL"):
            pass
        else:
            yield Tok(kind, text, pos)
        pos = m.end()
    yield Tok("EOF","",pos)

In [ ]:
# -------------------------------------
# Recursive-descent parser with precedence
# Grammar (informal):
#   expr   ::= let ID '=' expr 'in' expr
#            | if expr 'then' expr 'else' expr
#            | equality
#   equality ::= add ('==' add)*
#   add    ::= mul (('+' | '-') mul)*
#   mul    ::= primary ('*' primary)*
#   primary::= INT | TRUE | FALSE | ID | '(' expr ')'
# -------------------------------------

class ParseError(Exception): ...

class Parser:
    def __init__(self, toks):
        self.toks = list(toks)
        self.i = 0

    def cur(self): return self.toks[self.i]
    def eat(self, typ):
        if self.cur().typ == typ:
            self.i += 1
            return self.toks[self.i-1]
        raise ParseError(f"expected {typ} at {self.cur().pos}, found {self.cur().typ}")

    def match(self, *types):
        if self.cur().typ in types:
            t = self.cur(); self.i += 1; return t
        return None

    def parse(self):  # entry
        e = self.parse_expr()
        if self.cur().typ != "EOF":
            raise ParseError(f"extra input at {self.cur().pos}")
        return e

    def parse_expr(self):
        if self.cur().typ == "LET":
            self.eat("LET")
            name = self.eat("ID").val
            self.eat("ASSIGN")
            e1 = self.parse_expr()
            self.eat("IN")
            e2 = self.parse_expr()
            return Let(name, e1, e2)
        if self.cur().typ == "IF":
            self.eat("IF")
            c = self.parse_expr()
            self.eat("THEN")
            t = self.parse_expr()
            self.eat("ELSE")
            e = self.parse_expr()
            return If(c, t, e)
        return self.parse_equality()

    def parse_equality(self):
        e = self.parse_add()
        while self.match("EQ"):
            rhs = self.parse_add()
            e = Eq(e, rhs)
        return e

    def parse_add(self):
        e = self.parse_mul()
        while True:
            if self.match("PLUS"):
                e = Add(e, self.parse_mul())
            elif self.match("MINUS"):
                e = Sub(e, self.parse_mul())
            else:
                break
        return e

    def parse_mul(self):
        e = self.parse_primary()
        while self.match("TIMES"):
            e = Mul(e, self.parse_primary())
        return e

    def parse_primary(self):
        t = self.cur()
        if t.typ == "INT":
            self.eat("INT"); return Int(int(t.val))
        if t.typ == "TRUE":
            self.eat("TRUE"); return Bool(True)
        if t.typ == "FALSE":
            self.eat("FALSE"); return Bool(False)
        if t.typ == "ID":
            self.eat("ID"); return Var(t.val)
        if t.typ == "LP":
            self.eat("LP"); e = self.parse_expr(); self.eat("RP"); return e
        raise ParseError(f"unexpected token {t.typ} at {t.pos}")

In [ ]:
# -------------------------------------
# Interpreter (big-step)
# -------------------------------------
from typing import Dict, Union
Value = Union[int, bool]
Env = Dict[str, Value]

class RuntimeErrorPL(Exception): ...

def eval_ast(e: Expr, env: Env | None = None) -> Value:
    env = {} if env is None else dict(env)
    match e:
        case Int(n):  return n
        case Bool(b): return b
        case Var(x):
            if x in env: return env[x]
            raise RuntimeErrorPL(f"unbound {x}")
        case Add(a,b): return eval_ast(a,env) + eval_ast(b,env)
        case Sub(a,b): return eval_ast(a,env) - eval_ast(b,env)
        case Mul(a,b): return eval_ast(a,env) * eval_ast(b,env)
        case Eq(a,b):  return eval_ast(a,env) == eval_ast(b,env)
        case If(c,t,e2): return eval_ast(t,env) if eval_ast(c,env) else eval_ast(e2,env)
        case Let(x,e1,e2):
            v1 = eval_ast(e1,env); env2 = dict(env); env2[x]=v1; return eval_ast(e2,env2)

In [ ]:
# -------------------------------------
# Unit tests
# -------------------------------------
def parse(s: str) -> Expr:
    return Parser(tokenize(s)).parse()

def run(s: str) -> Value:
    return eval_ast(parse(s), {})

# Basic arithmetic & precedence
assert pretty(parse("1 + 2 * 3")) == "(1 + (2 * 3))"
assert pretty(parse("(1 + 2) * 3")) == "((1 + 2) * 3)"
assert run("1 + 2 * 3") == 7

# Equality and booleans
assert run("1 == 1") is True
assert run("1 == 2") is False
assert run("if true then 1 else 0") == 1

# let-binding
assert run("let x = 2 in x * (3 + 4)") == 14

print("ok  - Week 2 parser/interpreter basic tests passed")

### 🧪 Your Turn (15–20 minutes)
1) Extend the grammar with **unary negation** so `-3 * 2` parses as `(-3) * 2` (hint: add a `NEG` case in `parse_primary`).  
2) Improve error messages to include **line/column** information (track columns in `tokenize`).  
3) Add logical operators `and` / `or` with the usual **short‑circuiting** (parse to nested `if`s or add nodes).

### ✍️ Reflection (2–3 sentences)
- Where does the parser enforce **precedence** and **associativity**?  
- How do **scope** and **environment** interact in `let` evaluation?

In [ ]:
# 📝 Save small submission bundle
import json, time
stamp = time.strftime("%Y-%m-%d %H:%M:%S")
submission = {
  "student_name": STUDENT_NAME,
  "student_id": STUDENT_ID,
  "timestamp": stamp,
  "checks": ["precedence", "if-then-else", "let"],
  "reflection": "(fill in here)"
}
with open("week2_submission.json", "w") as f:
  json.dump(submission, f, indent=2)
print("Saved week2_submission.json — upload with your notebook.")